# 🏥 The Grumpy Doctogres: EfficientNet-B3 + AutoAugment (Windows Fix)

## 🛠️ Windows Stability Fix Applied
- **`num_workers=0`**: Windows cannot handle multi-process data loading easily inside Jupyter Notebooks. We have disabled worker processes to prevent the `DataLoader worker exited unexpectedly` crash. 
- **`AutoAugment`**: Maintained as the primary augmentation strategy.
- **`fit()` function**: Restored the exact robust training loop from your lectures.

In [33]:
# --------------------------------------------------------
# 1. IMPORTS & SETUP
# --------------------------------------------------------
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
from tqdm import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import time

# Set seed
SEED = 42
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.benchmark = True

set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [ ]:
# --------------------------------------------------------
# 2. DATA PREP & AUTO AUGMENTATION
# --------------------------------------------------------

# PATHS
DATASET_ROOT = os.path.join(os.path.pardir, "an2dl2526c2") 
LABELS_CSV = os.path.join(DATASET_ROOT, "train_labels.csv")
CLEAN_DATA_DIR = os.path.join(DATASET_ROOT, "train_masked_noshreks")

# HYPERPARAMETERS
BATCH_SIZE = 16 # RTX 3070 8GB VRAM Safe limit for B3
LEARNING_RATE = 1e-4 
EPOCHS = 100
PATIENCE = 15
IMG_SIZE = 300 # EfficientNet B3 Resolution

# Load Data Logic
labels_df = pd.read_csv(LABELS_CSV)
labels_map = dict(zip(labels_df.iloc[:, 0], labels_df.iloc[:, 1]))
valid_filenames = sorted([f for f in os.listdir(CLEAN_DATA_DIR) if f in labels_map])
y_labels_raw = [labels_map[f] for f in valid_filenames]

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_labels_raw)
CLASSES = label_encoder.classes_

# Split
X_train, X_temp, y_train, y_temp = train_test_split(valid_filenames, y_encoded, test_size=0.2, stratify=y_encoded, random_state=SEED)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=SEED)

# Class Weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

# Dataset Class
class HistologyDataset(Dataset):
    def __init__(self, filenames, labels, root_dir, transform=None):
        self.filenames = filenames
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self): return len(self.filenames)
    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.root_dir, self.filenames[idx])).convert("RGB")
        if self.transform: img = self.transform(img)
        return img, self.labels[idx]

# --- ⚡ AUTO AUGMENTATION ⚡ ---
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Loaders - num_workers=0 is CRITICAL for Windows stability in Notebooks
train_loader = DataLoader(HistologyDataset(X_train, y_train, CLEAN_DATA_DIR, train_transform), 
                          batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(HistologyDataset(X_val, y_val, CLEAN_DATA_DIR, val_transform), 
                        batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
test_loader = DataLoader(HistologyDataset(X_test, y_test, CLEAN_DATA_DIR, val_transform), 
                         batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

In [35]:
# --------------------------------------------------------
# 3. TRAINING FUNCTIONS (EXACTLY AS REQUESTED)
# --------------------------------------------------------

def train_one_epoch(model, train_loader, criterion, optimizer, scaler, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_targets = []
    
    # Iterate through training batches
    for inputs, targets in tqdm(train_loader, desc="Training", leave=False):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass with mixed precision
        with torch.amp.autocast(device_type=device.type, enabled=(device.type == 'cuda')):
            logits = model(inputs)
            loss = criterion(logits, targets)
            
        # Backward pass with gradient scaling
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # Update metrics
        running_loss += loss.item() * inputs.size(0)
        predictions = logits.argmax(dim=1)
        all_preds.extend(predictions.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())
        
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = accuracy_score(all_targets, all_preds)
    
    return epoch_loss, epoch_acc

def validate_one_epoch(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            with torch.amp.autocast(device_type=device.type, enabled=(device.type == 'cuda')):
                logits = model(inputs)
                loss = criterion(logits, targets)
            
            running_loss += loss.item() * inputs.size(0)
            predictions = logits.argmax(dim=1)
            all_preds.extend(predictions.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            
    epoch_loss = running_loss / len(val_loader.dataset)
    epoch_acc = accuracy_score(all_targets, all_preds)
    
    return epoch_loss, epoch_acc

def fit(model, train_loader, val_loader, epochs, criterion, optimizer, scaler, device, patience=10, model_name="model"):
    training_history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    best_val_acc = 0.0
    patience_counter = 0
    best_model_path = f"{model_name}_best.pth"
    
    print(f"Training {model_name} for {epochs} epochs on {device}...")
    
    for epoch in range(epochs):
        start_time = time.time()
        
        # Train
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, scaler, device)
        
        # Validate
        val_loss, val_acc = validate_one_epoch(model, val_loader, criterion, device)
        
        # Update History
        training_history['train_loss'].append(train_loss)
        training_history['train_acc'].append(train_acc)
        training_history['val_loss'].append(val_loss)
        training_history['val_acc'].append(val_acc)
        
        # Step Scheduler
        if 'scheduler' in globals():
            scheduler.step(val_loss)
            
        elapsed = time.time() - start_time
        
        print(f"Epoch {epoch+1}/{epochs} | Time: {elapsed:.0f}s")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
        
        # Early Stopping & Saving
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  --> New Best Model Saved! (Acc: {val_acc:.4f})")
        else:
            patience_counter += 1
            print(f"  --> Patience: {patience_counter}/{patience}")
            
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break
            
    return training_history

In [36]:
# --------------------------------------------------------
# 4. MODEL INIT & EXECUTION
# --------------------------------------------------------

MODEL_NAME = 'efficientnet_b3'
model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=len(CLASSES))
model = model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# Initialize Scaler for Mixed Precision
scaler = torch.cuda.amp.GradScaler(enabled=(device.type == 'cuda'))

# RUN THE FIT FUNCTION
history = fit(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=EPOCHS,
    criterion=criterion,
    optimizer=optimizer,
    scaler=scaler,
    device=device,
    patience=PATIENCE,
    model_name="efficientnet_b3_autoaugment"
)

e:\miniconda3\envs\an2dl-kaggle\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\Karim Negm\AppData\Local\Temp\ipykernel_8796\1369748484.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == 'cuda'))


Training efficientnet_b3_autoaugment for 100 epochs on cuda...


Epoch 1/100 | Time: 26s
  Train Loss: 2.3041 | Train Acc: 0.2507
  Val Loss:   2.0922 | Val Acc:   0.2540
  --> New Best Model Saved! (Acc: 0.2540)


Epoch 2/100 | Time: 12s
  Train Loss: 1.7391 | Train Acc: 0.3053
  Val Loss:   1.9479 | Val Acc:   0.3333
  --> New Best Model Saved! (Acc: 0.3333)


Epoch 3/100 | Time: 12s
  Train Loss: 1.5530 | Train Acc: 0.3330
  Val Loss:   1.9740 | Val Acc:   0.1825
  --> Patience: 1/15


...

Epoch 24/100 | Time: 15s
  Train Loss: 0.6207 | Train Acc: 0.7592
  Val Loss:   1.7765 | Val Acc:   0.3413
  --> Patience: 11/15


Epoch 25/100 | Time: 14s
  Train Loss: 0.5911 | Train Acc: 0.7820
  Val Loss:   1.7352 | Val Acc:   0.3175
  --> Patience: 12/15


Epoch 26/100 | Time: 15s
  Train Loss: 0.6169 | Train Acc: 0.7562
  Val Loss:   1.7686 | Val Acc:   0.3095
  --> Patience: 13/15


Epoch 27/100 | Time: 15s
  Train Loss: 0.6219 | Train Acc: 0.7572
  Val Loss:   1.8253 | Val Acc:   0.3095
  --> Patience: 14/15


Epoch 28/100 | Time: 15s
  Train Loss: 0.6358 | Train Acc: 0.7532
  Val Loss:   1.8640 | Val Acc:   0.2698
  --> Patience: 15/15
Early stopping triggered.


In [39]:
# --------------------------------------------------------
# 5. FINAL EVALUATION
# --------------------------------------------------------
model.load_state_dict(torch.load("efficientnet_b3_autoaugment_best.pth"))

test_loss, test_acc = validate_one_epoch(model, test_loader, criterion, device)
print(f"\nTest Accuracy: {test_acc:.4f}")

model.eval()
all_preds = []
all_targets = []
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions = outputs.argmax(dim=1)
        all_preds.extend(predictions.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

print(classification_report(all_targets, all_preds, target_names=CLASSES))

C:\Users\Karim Negm\AppData\Local\Temp\ipykernel_8796\1941345601.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("efficientnet_b3_autoau


Test Accuracy: 0.3071
                 precision    recall  f1-score   support

        HER2(+)       0.38      0.24      0.30        33
      Luminal A       0.24      0.44      0.31        34
      Luminal B       0.50      0.22      0.31        45
Triple negative       0.26      0.40      0.32        15

       accuracy                           0.31       127
      macro avg       0.34      0.33      0.31       127
   weighted avg       0.37      0.31      0.31       127



In [ ]:
# --------------------------------------------------------
# GENERATE KAGGLE SUBMISSION
# --------------------------------------------------------
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from PIL import Image

# 1. SETUP PATHS
# Adjust this if your test images are in a different folder name
TEST_DATA_DIR = os.path.join(DATASET_ROOT, "test_masked_noshreks") 

# Check if path exists
if not os.path.exists(TEST_DATA_DIR):
    print(f"⚠️ WARNING: Could not find '{TEST_DATA_DIR}'.")
    print("Please check where you extracted 'test_data.zip' and update TEST_DATA_DIR.")
else:
    print(f"Test Data found at: {TEST_DATA_DIR}")

# 2. DEFINE SUBMISSION DATASET
# (Does not require labels, returns image + filename)
class SubmissionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        # Get all valid image files
        self.filenames = sorted([
            f for f in os.listdir(root_dir) 
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff'))
        ])
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        fname = self.filenames[idx]
        img_path = os.path.join(self.root_dir, fname)
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
            
        return image, fname

# 3. PREPARE LOADER
# Use the same validation transform (Resize + Normalize only)
submission_dataset = SubmissionDataset(TEST_DATA_DIR, transform=val_transform)
# num_workers=0 for Windows stability
submission_loader = DataLoader(submission_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"Found {len(submission_dataset)} test images.")

# 4. LOAD BEST MODEL
model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=len(CLASSES))
model.load_state_dict(torch.load(f"efficientnet_b3_autoaugment_best.pth"))
model = model.to(device)
model.eval()

# 5. INFERENCE LOOP
all_filenames = []
all_predictions = []

print("Running Inference on Test Set...")
with torch.no_grad():
    for images, filenames in tqdm(submission_loader):
        images = images.to(device)
        
        # Forward pass
        outputs = model(images)
        
        # Get predicted class index
        _, preds = torch.max(outputs, 1)
        
        # Store results
        all_filenames.extend(filenames)
        all_predictions.extend(preds.cpu().numpy())

# 6. DECODE LABELS & SAVE CSV
# Convert numerical indices (0, 1, 2...) back to strings ('Luminal A', 'HER2+', etc.)
predicted_labels = label_encoder.inverse_transform(all_predictions)

# Create DataFrame
submission_df = pd.DataFrame({
    'sample_index': all_filenames,
    'label': predicted_labels
})

# Save to CSV
submission_df.to_csv("submission.csv", index=False)

print("\n✅ submission.csv generated successfully!")
print(submission_df.head())

Test Data found at: ..\an2dl2526c2\test_masked_noshreks
Found 954 test images.


C:\Users\Karim Negm\AppData\Local\Temp\ipykernel_8796\844993284.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"efficientnet_b3_autoa

Running Inference on Test Set...


100%|██████████| 60/60 [00:05<00:00, 10.38it/s]


✅ submission.csv generated successfully!
       filename            label
0  img_0000.png          HER2(+)
1  img_0001.png  Triple negative
2  img_0002.png        Luminal A
3  img_0003.png        Luminal A
4  img_0004.png          HER2(+)
